### Import statements

In [13]:
# numpy is a powerful data scientific computing library
import numpy as np
# matplotlib is great for data visualization
import matplotlib.pyplot as plt
%matplotlib notebook

### Import accoustic data as a 262144 x 49 matrix

d = np.load('subdata.npy')

### Define the length of the acoustic data cube, along with the Fourier domain

In [15]:
#length of cube representing acoustic data
N_grid = 64
#Defining the discretization of the Fourier space
k_grid = np.linspace(-N_grid/2, N_grid/2-1, N_grid)
#This 2D meshgrid allows us to visualize the Fourier space
k1, k2 = np.meshgrid(k_grid, k_grid)

## Sum and average the FFT's to find central frequency

In [16]:
fft = np.zeros((N_grid,N_grid,N_grid))
# sum each FFT for each timestep (0-48)
for t in np.arange(49):
    signal = np.reshape(d[:, t], (N_grid, N_grid, N_grid))
    
    fft = fft + np.fft.fftshift(np.fft.fftn(signal))
    

In [17]:
# Take the average    
fft_ave_real = np.abs(np.real(fft/49))
fft_ave = np.abs(fft/49)
max_indices_real = np.unravel_index(fft_ave_real.argmax(), fft_ave.shape)
zlev_real = max_indices_real[2]
max_indices = np.unravel_index(fft_ave.argmax(), fft_ave.shape)
zlev = max_indices[2]

#### Find the central frequency in the Fourier Space

In [18]:
print(max_indices)
print(max_indices_real)
print(max_indices - np.array([32,32,32]))
print(max_indices_real- np.array([32,32,32]))
print(zlev_real)
print(zlev)

(39, 49, 10)
(26, 17, 55)
[  7  17 -22]
[ -6 -15  23]
55
10


fig, ax= plt.subplots()
#ax.set_title('Average of the FFT at each timestep (0-48) z = 23')
ax.set_xlabel('kx')
ax.set_ylabel('ky')
im = plt.contourf(k1, k2, np.abs(fft_ave[:,:,zlev_real]))
fig.colorbar(im, label = "Fourier Coefficient")

fig1, ax1= plt.subplots()
#ax1.set_title('Average of the FFT at each timestep (0-48) z = -22')
ax1.set_xlabel('kx')
ax1.set_ylabel('ky')
im1 = plt.contourf(k1, k2, np.abs(fft_ave[:,:,10]))
fig1.colorbar(im1, label = "Fourier Coefficient")

## Define Gaussian filter around central frequency and shifted for k_grid

In [21]:
# Create a Gaussian filter centered on the central frequency found above
def g_centered(x, y, z, s):
    centered_x = x - max_indices_real[1] + 32
    centered_y = y - max_indices_real[0] + 32
    centered_z = z - max_indices_real[2] + 32
    result = np.exp(-((centered_x**2 + centered_y**2 
                    + centered_z**2)/(2*s**2)))
    return result

In [22]:
# Create a Gaussian filter centered on the central frequency found above
def g_centered_2(x, y, z, s):
    centered_x = x - max_indices[1] + 32
    centered_y = y - max_indices[0] + 32
    centered_z = z - max_indices[2] + 32
    result = np.exp(-((centered_x**2 + centered_y**2 
                    + centered_z**2)/(2*s**2)))
    return result

In [23]:
# set Sigma and define a 3D mesh to plot results
sigma = 3
kx3, ky3, kz3 = np.meshgrid(k_grid, k_grid, k_grid)

In [24]:
# Create the Gaussian filter values using the g_centered function defined previously
g_vals = g_centered(kx3, ky3, kz3, sigma)
g_vals_2 = g_centered_2(kx3, ky3, kz3, sigma)
total_g_vals = g_vals + g_vals_2

In [25]:
total_g_vals[26,17,55]

1.0

In [26]:
total_g_vals[39,49,10]

1.0

fig2, (ax2) = plt.subplots()
#ax2.set_title('Gaussian Filter at sigma=10 z=23')
ax2.set_xlabel('kx')
ax2.set_ylabel('ky')
im2 = plt.contourf(k1, k2, np.abs(total_g_vals[:,:, zlev_real]))
fig2.colorbar(im2, label = "Gaussian Value")

fig2_1, (ax2_1) = plt.subplots()
#ax2_1.set_title('Gaussian Filter at sigma=10 z=-22')
ax2_1.set_xlabel('kx')
ax2_1.set_ylabel('ky')
im2_1 = plt.contourf(k1, k2, np.abs(total_g_vals[:,:, 10]))
fig2_1.colorbar(im2_1, label = "Gaussian Value")

In [29]:
#Find the indices associated with the maximum value
np.unravel_index(g_vals.argmax(), g_vals.shape)

(26, 17, 55)

In [30]:
#Define a function which performs the FFT, filtering, and IFFT of each timestep
def denoise(total_g_vals, timestep):
    fft = np.fft.fftshift(np.fft.fftn(timestep))
    
    fft_denoise = fft * total_g_vals
    
    f_clean_vals = np.real(np.fft.ifftn( np.fft.ifftshift( fft_denoise )))
    
    return f_clean_vals

In [31]:

# Create and fill a new matrix to hold the denoised data
denoised_f = np.zeros((64,64,64,49))

for i in range(49):
    signal = np.reshape(d[:,i], (64,64,64))
    
    denoised_f[:,:,:,i] = denoise(total_g_vals,signal)
    

In [32]:
time_0 = denoised_f[:,:,:, 0]
zlev = np.unravel_index(time_0.argmax(), time_0.shape)
time_0 = time_0/time_0[zlev]
print(zlev[2])

32


N_grid = np.arange(N_grid)
n1, n2 = np.meshgrid(N_grid, N_grid)
fig3, ax3 = plt.subplots()
#ax3.set_title('Reconstructed Filtered Signal at t=0, z=32')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
im3 = plt.contourf(n1, n2, np.abs(time_0[:,:,zlev[2]]))
fig3.colorbar(im3)

In [34]:
time_24 = denoised_f[:,:,:, 24]
zlev = np.unravel_index(time_24.argmax(), time_24.shape)
time_24 = time_24/time_24[zlev]
print(zlev[2])

52


fig3_1, ax3_1 = plt.subplots()
#ax3_1.set_title('Reconstructed Filtered Signal at t=24, z=32')
ax3_1.set_xlabel('x')
ax3_1.set_ylabel('y')
im3_1 = plt.contourf(n1, n2, np.abs(time_24[:,:,zlev[2]]))
fig3_1.colorbar(im3_1)

In [36]:
time_48 = denoised_f[:,:,:, 48]
zlev = np.unravel_index(time_48.argmax(), time_48.shape)
time_48 = time_48/time_48[zlev]
print(zlev[2])

35


fig3_2, ax3_2 = plt.subplots()
#ax3_2.set_title('Reconstructed Filtered Signal at t=24, z=32')
ax3_2.set_xlabel('x')
ax3_2.set_ylabel('y')
im3_2 = plt.contourf(n1, n2, np.abs(time_48[:,:,zlev[2]]))
fig3_2.colorbar(im3_2)

In [38]:
# Determine the submarine location for each time step by taking the indices of the 
# maximum values of the filtered IFFT
sub_path = np.zeros((3,49))
for i in range(49):
    timestep = denoised_f[:,:,:,i]
    sub_path[:, i] = np.unravel_index(timestep.argmax(), timestep.shape)

fig4 = plt.figure()
ax4 = fig4.add_subplot(projection='3d')

ax4.plot3D(sub_path[1,:], sub_path[0,:], sub_path[2,:])
#ax4.set_title('Submarine Path')
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('z')


fig5 = plt.figure()
ax5 = fig5.subplots()

ax5.plot(sub_path[1,:], sub_path[0,:])
#ax5.set_title('Submarine Path')
ax5.set_xlabel('x')
ax5.set_ylabel('y')

In [41]:
#fig.savefig('Average_fft_z_23.png')
#fig1.savefig('Average_fft_z_neg22.png')
#fig2.savefig('Gauss_z_23.png')
#fig2_1.savefig('Gauss_z_neg22.png')
#fig3.savefig('Filterd_signal1.png')
#fig3_1.savefig('Filterd_signal2.png')
#fig3_2.savefig('Filterd_signal3.png')
#fig4.savefig('3D_path.png')
#fig5.savefig('2D_path.png')